In [1]:
import numpy as np
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch
from sklearn.model_selection import KFold
import wandb
import os

random.seed(27)
np.random.seed(27)
plt.style.use('ggplot')

In [2]:
from NN.denseLayer import *
from NN.loss import *
from NN.optimizer import *
from NN.activations import *

## Dataset

FashionMNIST.

In [3]:
num_classes = 10

In [4]:
def load_fashion_mnist(flatten=True):
    transform = transforms.ToTensor()

    train_ds = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=transform
    )

    test_ds = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=transform
    )

    # Convert to numpy
    X_train = train_ds.data.numpy().astype(np.float32)
    y_train = train_ds.targets.numpy()
    X_test = test_ds.data.numpy().astype(np.float32)
    y_test = test_ds.targets.numpy()

    # Normalize
    X_train /= 255.0
    X_test /= 255.0

    # Flatten 
    if flatten:
        X_train = X_train.reshape(len(X_train), -1)
        X_test = X_test.reshape(len(X_test), -1)
    
    return X_train, y_train, X_test, y_test

In [5]:
def one_hot(y, num_classes):
    Y = np.zeros((len(y), num_classes))
    Y[np.arange(len(y)), y] = 1
    return Y

def accuracy(probs, Y_true):
    preds = np.argmax(probs, axis=0)
    labels = np.argmax(Y_true, axis=0)
    return np.mean(preds == labels)

In [6]:
X_train, y_train, X_test, y_test = load_fashion_mnist(flatten=True) 

In [7]:
Y_train = one_hot(y_train, num_classes).T  # (10, 60000)
Y_test  = one_hot(y_test, num_classes).T   # (10, 10000)

print("X_train:", X_train.shape, "Y_train:", Y_train.shape)
print("X_test: ", X_test.shape,  "Y_test: ",  Y_test.shape)

X_train: (60000, 784) Y_train: (10, 60000)
X_test:  (10000, 784) Y_test:  (10, 10000)


## Sweep

Helper functions:

In [16]:
def get_activation_instance(name):
    if name == "relu":
        return ReLU()
    elif name == "tanh":
        return Tanh()
    elif name == "sigmoid":
        return Sigmoid()

def get_optimizer(name, lr):
    if name == "sgd":
        return SGD(learning_rate=lr)
    elif name == "adam":
        return Adam(learning_rate=lr)

Sweep configuration:

In [9]:
random_sweep = {
    "method": "random",
    "metric": {
        "name": "val_accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {"distribution": "log_uniform_values", "min": 0.0001, "max": 0.01},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd", "adam"]},
        "activation": {"values": ["relu", "tanh", "sigmoid"]},
        "num_classes": {"value": num_classes},
    },
}

In [10]:
bayes_sweep = {
    "method": "bayes",
    "metric": {"name": "mean_cv_accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"distribution": "log_uniform_values", "min": 0.0001, "max": 0.01},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 30},
        "num_hidden_layers": {"distribution": "int_uniform", "min": 1, "max": 5},
        "n_hidden_units": {"distribution": "int_uniform", "min": 32, "max": 128},
        "l2_coeff": {"distribution": "uniform", "min": 0.0, "max": 0.001},
        "optimizer": {"values": ["sgd", "adam"]},
        "activation": {"values": ["relu", "tanh", "sigmoid"]},
        "num_classes": {"value": num_classes},
    },
}

In [11]:
random_sweep_id = wandb.sweep(random_sweep, project="numpy-nn-random-search-cv")

Create sweep with ID: icx3j71h
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy-nn-random-search-cv/sweeps/icx3j71h


In [12]:
bayesian_sweep_id = wandb.sweep(sweep=bayes_sweep, project="numpy-nn-bayes-cv")

Create sweep with ID: i0egd2xn
Sweep URL: https://wandb.ai/xanderbaatz-danmarks-tekniske-universitet-dtu/numpy-nn-bayes-cv/sweeps/i0egd2xn


## Train procedure

In [12]:
#os.environ['WANDB_API_KEY'] = ...

wandb.login()

wandb: Currently logged in as: xanderbaatz (xanderbaatz-danmarks-tekniske-universitet-dtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
def stack_layers(input_dim, cfg):
    layers = []
    prev = input_dim

    # For all the layers (0, l-1)
    for _ in range(int(cfg['num_hidden_layers'])):
        act = get_activation_instance(cfg['activation'])
        layers.append(DenseLayer(prev, int(cfg['n_hidden_units']), activation=act,
                                 initializer=None, l2_coeff=float(cfg.get('l2_coeff', 0.0)))) #We set initialiser to none, since we have activation functions as hyperparatmerters, and denselayer takes care of what initialiser to use
        prev = int(cfg['n_hidden_units'])

    # Final layer
    layers.append(DenseLayer(prev, int(cfg['num_classes']), activation=Softmax(),
                             initializer=None))
    return layers

#Get the accuracy metric
def accuracy_from_probs(A, Y_true_onehot):
    preds = A.argmax(axis=0)
    truths = Y_true_onehot.argmax(axis=0)
    return float((preds == truths).mean())

def train_on_split(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, cfg):
    layers = stack_layers(X_train_fold.shape[1], cfg)
    optimizer = get_optimizer(cfg['optimizer'], float(cfg['learning_rate']))
    loss_fn = CrossEntropyLoss()

    m = X_train_fold.shape[0]
    epochs = int(cfg["epochs"])
    batch_size = int(cfg["batch_size"])

    epoch_loss = 0.0
    num_batches = 0

    #For each epoch, we perform batching, do the usual forward, backwards measure
    #again this is summarised in the FFN-explainer notebook.
    for epoch in range(epochs):
        perm = np.random.permutation(m)

        Xs = X_train_fold[perm].T    
        Ys = Y_train_fold[:, perm]    

        for i in range(0, m, batch_size):
            Xb = Xs[:, i:i+batch_size]
            Yb = Ys[:, i:i+batch_size]
            
            A = Xb
            for layer in layers:
                A = layer.forward(A)

            loss = loss_fn.forward(A, Yb)

            epoch_loss += loss
            num_batches += 1

            dA = loss_fn.backward(A, Yb)

            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)
        
    train_loss = epoch_loss / num_batches

    A_train = Xs
    for layer in layers:
        A_train = layer.forward(A_train)
    train_acc = accuracy_from_probs(A_train, Ys)

    A_val = X_val_fold.T
    for layer in layers:
        A_val = layer.forward(A_val)

    val_loss = loss_fn.forward(A_val, Y_val_fold)
    val_acc = accuracy_from_probs(A_val, Y_val_fold)

    wandb.log({
            "train_loss": float(train_loss),
            "train_accuracy": float(train_acc),
            "val_loss": float(val_loss),
            "val_accuracy": float(val_acc)
        })

    return val_acc, val_loss

def train_run():
    with wandb.init() as run:
        cfg = dict(run.config)
        run.log(dict(cfg))

        kf = KFold(n_splits=3, shuffle=True, random_state=27)
        fold_accuracies = []
        fold_val_losses = []

        #For N k-folds, train the split, and return the metrics
        for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train)):

            X_train_fold = X_train[train_idx]
            Y_train_fold = Y_train[:, train_idx]

            X_val_fold = X_train[val_idx]
            Y_val_fold = Y_train[:, val_idx]

            val_acc, val_loss = train_on_split(
                X_train_fold, Y_train_fold,
                X_val_fold, Y_val_fold,
                cfg
            )

            fold_accuracies.append(val_acc)
            fold_val_losses.append(val_loss)
            run.log({f"fold_{fold_idx+1}_accuracy": float(val_acc)})
            run.log({f"fold_{fold_idx+1}_loss": float(val_loss)})

        #Now we find the generlization error
        mean_cv_acc = float(np.mean(fold_accuracies))
        acc_cv_std = float(np.std(fold_accuracies))
        mean_cv_loss = float(np.mean(fold_val_losses))
        loss_cv_std = float(np.std(fold_val_losses))
        run.log({"mean_cv_accuracy": mean_cv_acc})
        run.log({"accuracy_cv_std": acc_cv_std})
        run.log({"mean_cv_loss": mean_cv_loss})
        run.log({"loss_cv_std": loss_cv_std})

        print(f"Run {run.id} \nCV Accuracies={fold_accuracies} \nMean={mean_cv_acc:.4f}")
        print(f"CV Losses={fold_val_losses} \nMean={mean_cv_loss:.4f}")

        wandb.finish()

In [16]:
# Bayesian
wandb.agent(bayesian_sweep_id, function=train_run, count=30)

wandb: Agent Starting Run: tf3krxy9 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	l2_coeff: 7.562229622513762e-05
wandb: 	learning_rate: 0.00010294482719454676
wandb: 	n_hidden_units: 103
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 5
wandb: 	optimizer: adam


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run tf3krxy9 
CV Accuracies=[0.88265, 0.8843, 0.88345] 
Mean=0.8835
CV Losses=[np.float64(0.33927505945984454), np.float64(0.33113124747259204), np.float64(0.331664947895519)] 
Mean=0.3340


accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...


wandb: Agent Starting Run: mfbl721x with config:
wandb: 	activation: tanh
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.0001762343345770151
wandb: 	learning_rate: 0.0003225147384664596
wandb: 	n_hidden_units: 121
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 2
wandb: 	optimizer: adam


...

accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r44e49ss with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 30
wandb: 	l2_coeff: 0.00022752476835996192
wandb: 	learning_rate: 0.00606526828608985
wandb: 	n_hidden_units: 125
wandb: 	num_classes: 10
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: adam


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run r44e49ss 
CV Accuracies=[0.8533, 0.85755, 0.85005] 
Mean=0.8536
CV Losses=[np.float64(0.41111387882493483), np.float64(0.39229848819569163), np.float64(0.42896995698731233)] 
Mean=0.4108


accuracy_cv_std,▁
batch_size,▁
epochs,▁
fold_1_accuracy,▁
fold_1_loss,▁
fold_2_accuracy,▁
fold_2_loss,▁
fold_3_accuracy,▁
fold_3_loss,▁
l2_coeff,▁
+11,...


## Best hyperparameters

In [ ]:
def train_test_epochs(X_train, Y_train, X_test, Y_test, best_cfg, seeds=10):

    epochs = int(best_cfg["epochs"])
    batch_size = int(best_cfg["batch_size"])

    m = X_train.shape[0]
    num_classes = Y_train.shape[0]

    all_train_losses = np.zeros((seeds, epochs))
    all_train_accs   = np.zeros((seeds, epochs))
    all_test_losses  = np.zeros((seeds, epochs))
    all_test_accs    = np.zeros((seeds, epochs))

    for seed in range(seeds):

        np.random.seed(seed)

        # new model per seed
        layers = stack_layers(X_train.shape[1], best_cfg)
        optimizer = get_optimizer(best_cfg["optimizer"],
                                  float(best_cfg["learning_rate"]))
        loss_fn = CrossEntropyLoss()

        # per seed sesh
        run = wandb.init(
            project="fashionmnist_final_epochs",
            group="10_seed_runs",       
            config=best_cfg,
            reinit=True,
            name=f"seed_{seed}"
        )

        for epoch in range(epochs):

            # everyday I'm shuffling
            perm = np.random.permutation(m)
            Xs = X_train[perm].T
            Ys = Y_train[:, perm]

            # for minibatching
            for i in range(0, m, batch_size):
                Xb = Xs[:, i:i+batch_size]
                Yb = Ys[:, i:i+batch_size]

                # forward
                A = Xb
                for layer in layers:
                    A = layer.forward(A)

                # backward
                dA = loss_fn.backward(A, Yb)
                for layer in reversed(layers):
                    dA, dW, db = layer.backward(dA)
                    optimizer.update(layer, dW, db)

            # Train
            A_train = X_train.T
            for layer in layers:
                A_train = layer.forward(A_train)

            train_loss = loss_fn.forward(A_train, Y_train)
            train_acc = accuracy_from_probs(A_train, Y_train)

            # Test
            A_test = X_test.T
            for layer in layers:
                A_test = layer.forward(A_test)

            test_loss = loss_fn.forward(A_test, Y_test)
            test_acc = accuracy_from_probs(A_test, Y_test)

            all_train_losses[seed, epoch] = train_loss
            all_train_accs[seed, epoch] = train_acc
            all_test_losses[seed, epoch] = test_loss
            all_test_accs[seed, epoch] = test_acc

            run.log({
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "test_loss": test_loss,
                "test_acc": test_acc
            })

            print(f"seed {seed} | epoch {epoch} | train Acc={train_acc:.4f} | test Acc={test_acc:.4f}")

        run.finish()

    metrics = {
        "train_loss_mean": np.mean(all_train_losses, axis=0),
        "train_loss_std":  np.std(all_train_losses, axis=0),
        "train_acc_mean":  np.mean(all_train_accs,   axis=0),
        "train_acc_std":   np.std(all_train_accs,    axis=0),
        "test_loss_mean":  np.mean(all_test_losses,  axis=0),
        "test_loss_std":   np.std(all_test_losses,   axis=0),
        "test_acc_mean":   np.mean(all_test_accs,    axis=0),
        "test_acc_std":    np.std(all_test_accs,     axis=0),
    }

    run = wandb.init(
        project="fashionmnist_final_epochs",
        name="aggregated_stats",
        config=best_cfg,
        group="10_seed_aggregate",
        reinit=True,
    )

    for epoch in range(epochs):
        run.log({
            "epoch": epoch,
            "train_loss_mean": metrics["train_loss_mean"][epoch],
            "train_loss_std":  metrics["train_loss_std"][epoch],
            "train_acc_mean":  metrics["train_acc_mean"][epoch],
            "train_acc_std":   metrics["train_acc_std"][epoch],
            "test_loss_mean":  metrics["test_loss_mean"][epoch],
            "test_loss_std":   metrics["test_loss_std"][epoch],
            "test_acc_mean":   metrics["test_acc_mean"][epoch],
            "test_acc_std":    metrics["test_acc_std"][epoch],
        })

    run.finish()
    return metrics


In [18]:
best_cfg = {
    "num_hidden_layers": 2,
    "n_hidden_units": 119,
    "activation": "tanh",
    "optimizer": "adam",
    "learning_rate": 0.001236,
    "batch_size": 128,
    "epochs": 100,
    "l2_coeff": 0.00005466,
    "num_classes": 10
}

In [19]:
train_test_epochs(X_train, Y_train, X_test, Y_test, best_cfg, seeds=10)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


[Seed 0] Epoch 0 | Train Acc=0.8648, Test Acc=0.8475
[Seed 0] Epoch 1 | Train Acc=0.8740, Test Acc=0.8546
[Seed 0] Epoch 2 | Train Acc=0.8876, Test Acc=0.8673
[Seed 0] Epoch 3 | Train Acc=0.8984, Test Acc=0.8773
[Seed 0] Epoch 4 | Train Acc=0.9015, Test Acc=0.8749
[Seed 0] Epoch 5 | Train Acc=0.9007, Test Acc=0.8730
[Seed 0] Epoch 6 | Train Acc=0.8978, Test Acc=0.8693
[Seed 0] Epoch 7 | Train Acc=0.9101, Test Acc=0.8850
[Seed 0] Epoch 8 | Train Acc=0.9087, Test Acc=0.8759
[Seed 0] Epoch 9 | Train Acc=0.9162, Test Acc=0.8803
[Seed 0] Epoch 10 | Train Acc=0.9142, Test Acc=0.8807
[Seed 0] Epoch 11 | Train Acc=0.9160, Test Acc=0.8804
[Seed 0] Epoch 12 | Train Acc=0.9253, Test Acc=0.8832
[Seed 0] Epoch 13 | Train Acc=0.9046, Test Acc=0.8693
[Seed 0] Epoch 14 | Train Acc=0.9246, Test Acc=0.8837
[Seed 0] Epoch 15 | Train Acc=0.9283, Test Acc=0.8874
[Seed 0] Epoch 16 | Train Acc=0.9316, Test Acc=0.8855
[Seed 0] Epoch 17 | Train Acc=0.9323, Test Acc=0.8866
[Seed 0] Epoch 18 | Train Acc=0.9326, 

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


[Seed 0] Epoch 99 | Train Acc=0.9799, Test Acc=0.8873


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇██
test_acc,▁▂▄▅▅▆▇▄▇▇▇▆▇▇▆▇▆▇▇▇▇▇▆▇▇▇▆▆█▇▇▇▆█▇█▇▇▇▇
test_loss,▄▂▁▁▃▂▁▁▁▁▁▂▂▂▁▂▃▂▃▄▂▄▃▃▃▃▄▅▅▅▄▆▅▆▅▇▇▇█▇
train_acc,▁▁▁▂▂▄▄▃▄▅▅▅▅▅▅▆▆▆▆▆▆▇▆▇▇▇▆▇▆▇▇▇▇▇▇█▇▇▇█
train_loss,█▇▆▅▅▆▅▅▅▄▄▄▃▃▄▃▃▅▂▃▃▂▂▂▃▂▃▁▂▂▁▁▁▁▁▁▁▁▂▁
epoch,99
test_acc,0.8873
test_loss,0.47991
train_acc,0.97992
train_loss,0.05597


...

[Seed 9] Epoch 99 | Train Acc=0.9774, Test Acc=0.8846


epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
test_acc,▁▃▄▆▅▆▇▆▆▇▇▇▇▆▆▆▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▆▆▇▇██▆▆▇
test_loss,▃▁▂▁▁▂▂▁▂▃▂▂▃▂▄▃▄▄▃▄▅▄▄▄▅▅▆▆▅▇▆▇▇▆▆▇▇▆▇█
train_acc,▁▁▂▃▃▄▄▄▅▅▆▅▅▆▆▆▆▇▇▇▆▆▇▇▇▇▇█▇▇▇▇▇▇██▇███
train_loss,█▇▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▃▂▃▂▃▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂▁▁▁
epoch,99
test_acc,0.8846
test_loss,0.5163
train_acc,0.97735
train_loss,0.06221


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇██
test_acc_mean,▁▃▄▅▆▇▇████████████████▇██████████▇█████
test_acc_std,▅▂▄▅▃▁▃▄▂▃▅▄▃▂▃▃▃▄▂▁▂▂▁▄▃▁▃▂▄▂▃▂▄▂▂▃▃▂█▂
test_loss_mean,▂▂▁▁▁▁▁▁▁▂▂▂▃▃▃▃▃▄▄▃▄▄▄▅▅▅▅▅▅▆▆▇▇▇█▇▇███
test_loss_std,▃▃▂▄▄▃▃▂▁▂▃▂▃▁▄▄█▄▅▃▄▃▅▄▅▂▄▄▇▅▅█▅▅▆▃▃▃▃▅
train_acc_mean,▁▂▂▃▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██▇█████████
train_acc_std,▆▃▃▂▃▃▂▇▃▁▂▃▃▁▃█▆▃▄▂▅▂▆▂▅▇▆▃▄▃▃▄▄▄▃▄▆▆▄▃
train_loss_mean,█▇▆▅▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
train_loss_std,▃▃▄▄▃▃▁▂▃▅▇▅▄▂▃█▂▃▄▅▂▂▂▂▄▄▇▇▃▃▅█▄▃▄▃▇▆▁▂
epoch,99
test_acc_mean,0.88583


{'train_loss_mean': array([0.37223365, 0.33862182, 0.30822025, 0.28975097, 0.27474248,
        0.26569415, 0.25781624, 0.25399087, 0.23813947, 0.23101515,
        0.21894224, 0.21677186, 0.21125218, 0.20516925, 0.19936902,
        0.19446575, 0.18612572, 0.18608912, 0.18102412, 0.1843872 ,
        0.17548015, 0.16952615, 0.16352395, 0.16220105, 0.16177781,
        0.15861634, 0.15111027, 0.15533506, 0.14685097, 0.14441332,
        0.14146569, 0.14292052, 0.13855243, 0.13921614, 0.1401023 ,
        0.12707493, 0.13542259, 0.12714115, 0.12866781, 0.11896575,
        0.11691779, 0.11956134, 0.11809825, 0.11333899, 0.11338114,
        0.10728507, 0.11148458, 0.11542416, 0.11271282, 0.10391527,
        0.10408421, 0.1007645 , 0.09988495, 0.09471638, 0.09466102,
        0.10147101, 0.09645981, 0.09165778, 0.08898652, 0.09314474,
        0.0918393 , 0.0952324 , 0.08745368, 0.09267161, 0.08399738,
        0.08408408, 0.08670649, 0.08683123, 0.08774102, 0.09121928,
        0.0840822 , 0.0800611

### Confusion matrix

In [ ]:
def train_test_confusion(X_train, Y_train, X_test, Y_test, best_cfg):
    """
    Train on X_train/Y_train and compute confusion matrices on X_test/Y_test per epoch.
    Single seed only.
    """
    import wandb

    epochs = int(best_cfg["epochs"])
    batch_size = int(best_cfg["batch_size"])
    m = X_train.shape[0]
    num_classes = Y_train.shape[0]

    test_confusions = []

    np.random.seed(0)

    layers = stack_layers(X_train.shape[1], best_cfg)
    optimizer = get_optimizer(best_cfg["optimizer"], float(best_cfg["learning_rate"]))
    loss_fn = CrossEntropyLoss()

    run = wandb.init(
        project="fashionmnist_final_conf",
        name="seed_0",
        config=best_cfg,
        reinit=True
    )

    for epoch in range(epochs):
        # shuffling
        perm = np.random.permutation(m)
        Xs = X_train[perm].T
        Ys = Y_train[:, perm]

        for i in range(0, m, batch_size):
            Xb = Xs[:, i:i+batch_size]
            Yb = Ys[:, i:i+batch_size]

            # forward
            A = Xb
            for layer in layers:
                A = layer.forward(A)

            # backward
            dA = loss_fn.backward(A, Yb)
            for layer in reversed(layers):
                dA, dW, db = layer.backward(dA)
                optimizer.update(layer, dW, db)

        A_test = X_test.T
        for layer in layers:
            A_test = layer.forward(A_test)
        y_pred_test = np.argmax(A_test, axis=0)  # (m,)
        y_true_test = Y_test.argmax(axis=0)      # (m,)

        cm = np.zeros((num_classes, num_classes), dtype=int)
        for t, p in zip(y_true_test, y_pred_test):
            cm[t, p] += 1
        test_confusions.append(cm)

        run.log({
            "epoch": epoch,
            "test_confusion": wandb.plot.confusion_matrix(
                y_true=y_true_test,
                preds=y_pred_test,
                title=f"Confusion Matrix (Epoch {epoch})"
            )
        })

        acc = np.mean(y_pred_test == y_true_test)
        print(f"Epoch {epoch} | Test Acc={acc:.4f}")

    run.finish()

    return test_confusions

confusions = train_test_confusion(X_train, Y_train, X_test, Y_test, best_cfg)

Epoch 0 | Test Acc=0.8475
Epoch 1 | Test Acc=0.8546
Epoch 2 | Test Acc=0.8673
Epoch 3 | Test Acc=0.8773
Epoch 4 | Test Acc=0.8749
Epoch 5 | Test Acc=0.8730
Epoch 6 | Test Acc=0.8693
Epoch 7 | Test Acc=0.8850
Epoch 8 | Test Acc=0.8759
Epoch 9 | Test Acc=0.8803
Epoch 10 | Test Acc=0.8807
Epoch 11 | Test Acc=0.8804
Epoch 12 | Test Acc=0.8832
Epoch 13 | Test Acc=0.8693
Epoch 14 | Test Acc=0.8837
Epoch 15 | Test Acc=0.8874
Epoch 16 | Test Acc=0.8855
Epoch 17 | Test Acc=0.8866
Epoch 18 | Test Acc=0.8873
Epoch 19 | Test Acc=0.8747
Epoch 20 | Test Acc=0.8813
Epoch 21 | Test Acc=0.8842
Epoch 22 | Test Acc=0.8865
Epoch 23 | Test Acc=0.8815
Epoch 24 | Test Acc=0.8858
Epoch 25 | Test Acc=0.8839
Epoch 26 | Test Acc=0.8882
Epoch 27 | Test Acc=0.8868
Epoch 28 | Test Acc=0.8788
Epoch 29 | Test Acc=0.8877
Epoch 30 | Test Acc=0.8853
Epoch 31 | Test Acc=0.8855
Epoch 32 | Test Acc=0.8914
Epoch 33 | Test Acc=0.8865
Epoch 34 | Test Acc=0.8856
Epoch 35 | Test Acc=0.8840
Epoch 36 | Test Acc=0.8828
Epoch 37 | 

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Epoch 99 | Test Acc=0.8873


epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
epoch,99
